# Reproduction of DECAF: Generating Fair Synthetic Data Using Causally-Aware Generative Networks

In this notebook we reproduce the results from the paper.

In [248]:
#from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

## Loading the Adult dataset

In [249]:
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "income",
]
def load_adult():
    """Load the Adult dataset in a pandas dataframe"""

    path = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    names = [
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "income",
    ]
    df = pd.read_csv(path, names=names, index_col=False)
    df = df.applymap(lambda x: x.strip() if type(x) is str else x)

    for col in df:
        if df[col].dtype == "object":
            df = df[df[col] != "?"]
    
    return df

In [250]:
# Display examples from the dataset
adult_dataset = load_adult()
adult_dataset.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [251]:
def load_train_test_data(df):
    """Get GAN training data from Adult dataset"""
    replace = [
        [
            "Private",
            "Self-emp-not-inc",
            "Self-emp-inc",
            "Federal-gov",
            "Local-gov",
            "State-gov",
            "Without-pay",
            "Never-worked",
        ],
        [
            "Bachelors",
            "Some-college",
            "11th",
            "HS-grad",
            "Prof-school",
            "Assoc-acdm",
            "Assoc-voc",
            "9th",
            "7th-8th",
            "12th",
            "Masters",
            "1st-4th",
            "10th",
            "Doctorate",
            "5th-6th",
            "Preschool",
        ],
        [
            "Married-civ-spouse",
            "Divorced",
            "Never-married",
            "Separated",
            "Widowed",
            "Married-spouse-absent",
            "Married-AF-spouse",
        ],
        [
            "Tech-support",
            "Craft-repair",
            "Other-service",
            "Sales",
            "Exec-managerial",
            "Prof-specialty",
            "Handlers-cleaners",
            "Machine-op-inspct",
            "Adm-clerical",
            "Farming-fishing",
            "Transport-moving",
            "Priv-house-serv",
            "Protective-serv",
            "Armed-Forces",
        ],
        [
            "Wife",
            "Own-child",
            "Husband",
            "Not-in-family",
            "Other-relative",
            "Unmarried",
        ],
        ["White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"],
        ["Female", "Male"],
        [
            "United-States",
            "Cambodia",
            "England",
            "Puerto-Rico",
            "Canada",
            "Germany",
            "Outlying-US(Guam-USVI-etc)",
            "India",
            "Japan",
            "Greece",
            "South",
            "China",
            "Cuba",
            "Iran",
            "Honduras",
            "Philippines",
            "Italy",
            "Poland",
            "Jamaica",
            "Vietnam",
            "Mexico",
            "Portugal",
            "Ireland",
            "France",
            "Dominican-Republic",
            "Laos",
            "Ecuador",
            "Taiwan",
            "Haiti",
            "Columbia",
            "Hungary",
            "Guatemala",
            "Nicaragua",
            "Scotland",
            "Thailand",
            "Yugoslavia",
            "El-Salvador",
            "Trinadad&Tobago",
            "Peru",
            "Hong",
            "Holand-Netherlands",
        ],
        [">50K", "<=50K"],
    ]

    for row in replace:
        df = df.replace(row, range(len(row)))

    df = df.values
    X = df[:, :14].astype(np.uint32)
    X = MinMaxScaler().fit_transform(X)
    for row in X:
        if row[9]>0:
            row[9] = 1
        elif row[9]<0:
            row[9] = 0
    y = df[:, 14].astype(np.uint8)

    return train_test_split(X, y, test_size=2000, stratify=y)

In [252]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

In [253]:
def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

In [254]:
# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(adult_dataset, dag)
print(dag_seed)

[[8, 6], [8, 14], [8, 12], [8, 3], [8, 5], [0, 6], [0, 12], [0, 14], [0, 1], [0, 5], [0, 3], [0, 7], [9, 6], [9, 5], [9, 14], [9, 1], [9, 3], [9, 7], [13, 5], [13, 12], [13, 3], [13, 1], [13, 14], [13, 7], [5, 6], [5, 12], [5, 14], [5, 1], [5, 7], [5, 3], [3, 6], [3, 12], [3, 14], [3, 1], [3, 7], [6, 14], [12, 14], [1, 14], [7, 14]]


In [255]:
bias_dict_ftu = create_bias_dict(adult_dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

bias_dict_dp = create_bias_dict(adult_dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

bias_dict_cf = create_bias_dict(adult_dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

Bias dict FTU: {14: [9]}
Bias dict DP: {14: [6, 12, 5, 3, 9, 1, 7]}
Bias dict CF: {14: [5, 9]}


In [256]:
# Get training and testing data
X_train, X_test, y_train, y_test = load_train_test_data(adult_dataset)

pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000,28162.000000
mean,0.293510,0.122429,0.119557,0.224993,0.608155,0.175502,0.364334,0.478340,0.111249,0.675485,0.011056,0.020284,0.407681,0.037904
std,0.180007,0.243230,0.071738,0.228722,0.170109,0.195299,0.229230,0.246044,0.299087,0.468202,0.074709,0.092881,0.122158,0.140956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.150685,0.000000,0.070643,0.066667,0.533333,0.000000,0.153846,0.400000,0.000000,0.000000,0.000000,0.000000,0.397959,0.000000
50%,0.273973,0.000000,0.111862,0.200000,0.600000,0.166667,0.307692,0.400000,0.000000,1.000000,0.000000,0.000000,0.397959,0.000000
75%,0.410959,0.166667,0.151757,0.266667,0.800000,0.333333,0.538462,0.600000,0.000000,1.000000,0.000000,0.000000,0.448980,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Evaluate on original dataset

In [257]:
# Train an MLP
mlp = MLPClassifier().fit(X_train, y_train)

In [258]:
def eval_model(mlp, x = X_test, y = y_test):
    """Helper function that prints evaluation metrics."""

    y_pred = mlp.predict(x[:, :14])

#     print('Precision:', precision_score(y, y_pred))
#     print('Recall:', recall_score(y, y_pred))
#     print('AUROC:', roc_auc_score(y, y_pred))
#     print('Accuracy:', accuracy_score(y, y_pred))
    prec = precision_score(y, y_pred)
    rec = recall_score(y, y_pred)
    roc = roc_auc_score(y, y_pred)
    acc = accuracy_score(y, y_pred)
    
    return prec, rec, roc, acc

def cal_DP(mlp, X_test):
    """Calculate fairness metric DP"""
    
    X_test_df = pd.DataFrame(X_test, columns = names[:-1]) 
    X_test_0 = X_test_df[X_test_df["sex"] <0.5]
    X_test_1 = X_test_df[X_test_df["sex"] >0.5]
    dp = abs(np.mean(mlp.predict(X_test_0)) - np.mean(mlp.predict(X_test_1)))
    
    #print("DP is: ",dp)
    return dp
    
    
# def FTU(X_test, X_synth, y_synth):
#     """Calculate fairness metric FTU"""
#     X_synth_0 = np.delete(X_synth,9,1) #Delete column of protected attribute
#     X_test_0 = np.delete(X_test,9,1) #Delete column of protected attribute
    
#     mlp_0 = MLPClassifier().fit(X_synth_0, y_synth)
#     mlp_1 = MLPClassifier().fit(X_synth, y_synth)
    
#     ftu = abs(mlp_0.predict_proba(X_test_0) - mlp_1.predict_proba(X_test))
    
#     print("FTU is: ", np.sum(ftu)/len(X_test))
    
def cal_FTU(mlp, X_test):
    """Calculate fairness metric DP"""
    
    X_test_df = pd.DataFrame(X_test, columns = names[:-1])
    X_test_0 = X_test_df.assign(sex = 0)
    X_test_1 = X_test_df.assign(sex = 1)

    ftu = abs(np.mean(mlp.predict(X_test_0)) - np.mean(mlp.predict(X_test_1)))
    
    #print("new_FTU is: ",ftu)
    return ftu

In [259]:
# Evaluate original data
y_pred = mlp.predict(X_test)

print(
    "precision: ", precision_score(y_test, y_pred),
    "recall: ", recall_score(y_test, y_pred),
    "roc_auc: ", roc_auc_score(y_test, y_pred),
)

print(cal_DP(mlp, X_test))
print(cal_FTU(mlp, X_test))
#FTU(X_test = X_test, X_synth = X_test, y_synth = y_test)

precision:  0.8726355611601513 recall:  0.9214380825565912 roc_auc:  0.757907796298376
0.20652317171145296
0.024499999999999966


## DECAF

### Train model

In [262]:
from models.DECAF import DECAF
from data import DataModule

train_data = np.column_stack((X_train, y_train))
dm = DataModule(train_data)

model = DECAF(
    dm.dims[0],
    dag_seed=dag_seed,
    h_dim=200,
    lr=0.5e-3,
    batch_size=64,
    lambda_privacy=0,
    lambda_gp=10,
    d_updates=10,
    alpha=2,
    rho=2,
    weight_decay=1e-2,
    grad_dag_loss=False,
    l1_g=0,
    l1_W=1e-4,
    p_gen=-1,
    use_mask=True,
)

trainer = pl.Trainer(max_epochs=50, logger=False)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | generator     | Generator_causal | 134 K 
1 | discriminator | Discriminator    | 43.6 K
---------------------------------------------------
178 K     Trainable params
225       Non-trainable params
178 K     Total params
0.713     Total estimated model params size (MB)


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0.

In [263]:
def generate_synthetic_data(biased_edges={}):
    """Generate synthetic data which is also optionally debiased."""
    X_synth = (
        model.gen_synthetic(
            dm.dataset.x,
            gen_order=model.get_gen_order(),
            biased_edges=biased_edges,
        )
        .detach()
        .numpy()
    )

    return X_synth[:, :14], np.rint(X_synth[:,14]).astype(np.uint8)

### Evaluate DECAF-ND

In [264]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data()
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)


# Train downstream model and eval
print("test on real data")
ndmlp = MLPClassifier().fit(X_synth_train, y_synth_train)

prec, rec, roc, acc = np.mean([eval_model(ndmlp) for i in range(1)], axis=0)
dp = np.mean([ cal_DP(ndmlp, X_test=X_synth) for i in range(1)])
ftu = np.mean([ cal_FTU(ndmlp, X_test=X_synth) for i in range(1)])
print("prec :",prec) 
print("rec :",rec)
print("roc :",roc) 
print("dp :",dp) 
print("ftu :",ftu) 
#dp = DP(ndmlp, X_test=X_synth)
#ftu = new_FTU(ndmlp, X_test=X_synth)


#FTU(X_test = X_test, X_synth = X_synth, y_synth = y_synth)
print("**************")
print("test on synth data")
# the weird thing they did in original code
#eval_model(ndmlp, X_synth_test, y_synth_test)
prec, rec, roc, acc = np.mean([eval_model(ndmlp, X_synth_test, y_synth_test) for i in range(3)], axis=0)
print("prec :",prec) 
print("rec :",rec)
print("roc :",roc)

test on real data
prec : 0.8112333526346265
rec : 0.9327563249001332
roc : 0.6390689255022755
dp : 0.09162694317038744
ftu : 0.03362687309139978
**************
test on synth data
prec : 0.7876905702992659
rec : 0.93
roc : 0.5890000000000001


In [265]:
y_synth = ndmlp.predict(X_synth)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

# Train downstream model and eval
ndmlp = MLPClassifier().fit(X_synth_train, y_synth_train)

print("test on real data")
p_r_r_a = [eval_model(ndmlp) for i in range(3)]
prec_mean, rec_mean, roc_mean, acc_mean = np.mean(p_r_r_a, axis=0)
prec_std, rec_std, roc_std, acc_std = np.std(p_r_r_a, axis=0)

dp = [ cal_DP(ndmlp, X_test=X_synth) for i in range(3) ]
dp_mean = np.mean(dp)
dp_std = np.std(dp)

ftu = [ cal_FTU(ndmlp, X_test=X_synth) for i in range(3)]
ftu_mean = np.mean(ftu)
ftu_std = np.std(ftu)

print("prec_mean :", prec, "prec_std :", prec_std) 
print("rec_mean :", rec_mean, "rec_std :", rec_std) 
print("roc_mean :", roc_mean, "roc_std :", roc_std) 
print("acc_mean :", acc_mean, "acc_std :", acc_std) 

print("dp_mean :", dp_mean, "dp_std :", dp_std)
print("ftu_mean :", ftu_mean, "ftu_std :", ftu_std)

print("**************")
print("test on synth data")
# the weird thing they did in original code
#eval_model(ndmlp, X_synth_test, y_synth_test)

prec, rec, roc, acc = np.mean([eval_model(ndmlp, X_synth_test, y_synth_test) for i in range(3)], axis=0)
print("prec :",prec) 
print("rec :",rec)
print("roc :",roc)

E:\Users\anaconda3\envs\py36\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


test on real data
prec_mean : 0.7876905702992659 prec_std : 0.0
rec_mean : 0.9454061251664448 rec_std : 1.1102230246251565e-16
roc_mean : 0.6413777613784031 roc_std : 0.0
acc_mean : 0.794 acc_std : 0.0
dp_mean : 0.08765577445982464 dp_std : 0.0
ftu_mean : 0.03174490448121581 ftu_std : 0.0
**************
test on synth data
prec : 0.9915062287655719
rec : 0.9915062287655719
roc : 0.963701832331504


### Evaluate DECAF-FTU

In [269]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_ftu)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

# Train downstream model and eval
ftumlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(ftumlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 
 

ftu = cal_FTU(ftumlp, X_test=X_synth)
dp = cal_DP(ftumlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)

# the weird thing they did in original code
prec, rec, roc, acc = eval_model(ftumlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7759914255091104
rec : 0.9640479360852197
roc : 0.5623452531831721
dp is:  0.035994555189879596
ftu is:  0.014239045522335014
prec : 0.7812160694896851
rec : 0.9479578392621871
roc : 0.5559291270999732


In [272]:
y_synth = ftumlp.predict(X_synth)# Generate synthetic data
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

ftumlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(ftumlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 
 

ftu = cal_FTU(ftumlp, X_test=X_synth)
dp = cal_DP(ftumlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)

# the weird thing they did in original code
prec, rec, roc, acc = eval_model(ftumlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7911111111111111
rec : 0.948069241011984
roc : 0.5965245803453494
dp is:  0.08916807833168294
ftu is:  0.044670122860592354
prec : 0.9994413407821229
rec : 0.9988833054159687
roc : 0.9970493082103766


### Evaluate DECAF-DP

In [280]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_dp)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)


# Train downstream model and eval
dpmlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(dpmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc_mean) 

ftu = cal_FTU(dpmlp, X_test=X_synth)
dp = cal_DP(dpmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)


# the weird thing they did in original code
prec, rec, roc, acc = eval_model(dpmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.750126454223571
rec : 0.9873501997336884
roc : 0.6413777613784031
dp is:  0.008325419801428757
ftu is:  0.004083516795682085
prec : 0.7626693426994481
rec : 0.9960681520314548
roc : 0.4990889283364025


In [282]:
y_synth = dpmlp.predict(X_synth)# Generate synthetic data
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

dpmlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(dpmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

ftu = cal_FTU(dpmlp, X_test=X_synth)
dp = cal_DP(dpmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)


# the weird thing they did in original code
prec, rec, roc, acc = eval_model(dpmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7503782148260212
rec : 0.9906790945406125
roc : 0.49835159546307733
dp is:  0.004342159983573901
ftu is:  0.0012073006178539014
prec : 0.9994987468671679
rec : 1.0
roc : 0.9166666666666667


### Evaluate DECAF-CF

In [278]:
# Generate synthetic data
X_synth, y_synth = generate_synthetic_data(biased_edges=bias_dict_cf)
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)


# Train downstream model and eval
cfmlp = MLPClassifier().fit(X_synth_train, y_synth_train)
eval_model(cfmlp)

prec, rec, roc, acc = eval_model(cfmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

ftu = cal_FTU(cfmlp, X_test=X_synth)
dp = cal_DP(cfmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)

# the weird thing they did in original code
prec, rec, roc, acc = eval_model(cfmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7656492498706674
rec : 0.9853528628495339
roc : 0.5378571543163332
dp is:  0.016329974763844235
ftu is:  0.015446346140188916
prec : 0.7696600710299341
rec : 0.9915032679738562
roc : 0.5127729105826728


In [279]:
y_synth = cfmlp.predict(X_synth)# Generate synthetic data
X_synth_train, X_synth_test, y_synth_train, y_synth_test = train_test_split(X_synth, y_synth, test_size=2000, stratify=y_synth)

cfmlp = MLPClassifier().fit(X_synth_train, y_synth_train)
prec, rec, roc, acc = eval_model(dpmlp)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

ftu = cal_FTU(dpmlp, X_test=X_synth)
dp = cal_DP(dpmlp, X_test=X_synth)
print("dp is: ", dp)
print("ftu is: ", ftu)


# the weird thing they did in original code
prec, rec, roc, acc = eval_model(dpmlp, X_synth_test, y_synth_test)
print("prec :", prec) 
print("rec :", rec) 
print("roc :", roc) 

prec : 0.7768331562167906
rec : 0.9733688415446072
roc : 0.5649976737843517
dp is:  0.01673258101112096
ftu is:  0.028123002627654214
prec : 0.9957424161788185
rec : 0.9492643328259767
roc : 0.8367011319302297


Fairness Calcs

Precision: 0.9334600760456274
Recall: 0.6537949400798935
AUROC: 0.7566163455419548
Accuracy: 0.705


NameError: name 'X' is not defined